In [1]:
import reg_mri
import os
from glob import glob
from utils import compute_mean_dice, getDevice
import nibabel as nib
from scipy.spatial.distance import dice
import numpy as np
import itk
import SimpleITK as sitk
import scipy.ndimage
import scipy
import matplotlib.pyplot as plt
from transforms_dict import getRegistrationEvalInverseTransformForMRI, SaveTransformForMRI
from tqdm import tqdm
import monai
import subprocess
from monai.networks.blocks import Warp
import torch

In [5]:
outfolders = ["scenario1",
              "scenario2",
              "scenario3",
             ]

models = [
    "scenario1_ddf+sym_1.0-0.0-8.0.pth",
    "scenario2_ddf+sym_1.0-0.0-8.0.pth",    
    "scenario3_ddf+sym_1.0-0.0-8.0.pth",
         ]
device = getDevice()
for i, model in enumerate(models):
    outdataset = 'GIN'    
    outfolder = outfolders[i]
    mris = sorted(glob(os.path.join('mubarak', "*_mri.nii.gz")))
    #labels = sorted(glob(os.path.join('dataset3', outdataset, 'Labels', "*_affine_convert.nii.gz")))
    warp_nearest = Warp("nearest", "zeros")
    atlas_name = "mubarak/atlasdown.nii.gz"    
    
    ants_warp = "mubarak/atlasdown.nii.gz"
    affine = nib.load(ants_warp).affine
    header = nib.load(ants_warp).header

    for i, mri in enumerate(mris):
        outname = "mubarak/" + model.split('.pth')[0] + "_" + mri.split('/')[-1].split('.')[0]
        newmodel = False
        sym = True
        pred_images, ddfs = reg_mri.main(model, mri, outname, False, True, "local", newmodel=newmodel, sym=sym)
        pred_image = pred_images[0]
        predsym_image = pred_images[1]

        moving_image = ddfs[0].cpu()
        deformable_ddf = ddfs[1].cpu()
        deformable_ddf = deformable_ddf.cpu().numpy()
        deformable_ddf = np.transpose(deformable_ddf, (2, 3, 4, 0, 1)) 
        deformable_ddf = deformable_ddf * [-1, -1, 1]
        deformable_image = nib.Nifti1Image(deformable_ddf, affine, header)
        
        moving_labels = torch.from_numpy(nib.load(labels[i]).get_fdata()).type('torch.FloatTensor').unsqueeze(0).unsqueeze(0)
        pred_labels = warp_nearest(moving_labels.to(device), ddfs[1]).cpu().numpy()       
        outname_label = "mubarak/" + model + "_" + mri.split('/')[-1].split('.')[0] + "_labels.nii.gz"
        labels_image = nib.Nifti1Image(pred_labels.squeeze(), affine, header)
        nib.save(labels_image, outname_label)
        print(outname_label)
        
        outname_deformable = "mubarak/" + model.split('.pth')[0] + "_" + mri.split('/')[-1].split('.')[0] + "_warp.nii.gz"
        nib.save(deformable_image, outname_deformable)
        print(outname_deformable)
        
        deformable_sym_ddf = ddfs[2].cpu()
        deformable_sym_ddf = deformable_sym_ddf.cpu().numpy()
        deformable_sym_ddf = np.transpose(deformable_sym_ddf, (2, 3, 4, 0, 1)) 
        deformable_sym_ddf = deformable_sym_ddf * [-1, -1, 1]
        deformable_sym_image = nib.Nifti1Image(deformable_sym_ddf, affine, header)
        outname_sym_deformable = "mubarak/" + model.split('.pth')[0] + "_" + mri.split('/')[-1].split('.')[0] + "_invwarp.nii.gz"
        nib.save(deformable_sym_image, outname_sym_deformable)
        

RuntimeError: applying transform <transforms.ResampleMRIToAtlas object at 0x7f1b00107fa0>

In [11]:
outfolders = ["scenario1",
              "scenario2",
              "scenario3",
             ]

models = [
    "scenario1_ddf+sym_1.0-0.0-8.0.pth",
    "scenario2_ddf+sym_1.0-0.0-8.0.pth",    
    "scenario3_ddf+sym_1.0-0.0-8.0.pth",
         ]
outdataset = 'GIN'    

for i, model in enumerate(models):
    outfolder = outfolders[i]
    mridir = 'output/' + outdataset + '/' + outfolder + '/'
    #affine_warps = sorted(glob(os.path.join(mridir, 'AffineWarp', "*.nii.gz")))
    deformable_warps = sorted(glob(os.path.join(mridir, 'DeformableWarp', "*_warp.nii.gz")))
    for i in range(len(deformable_warps)):
        #getLog10Jacobian_affine_ants(affine_warps[i], outfolder)    
        #print(deformable_warps[i])
        getLog10Jacobian_deformable_ants(deformable_warps[i], outfolder, outdataset)  

In [9]:
def getLog10Jacobian_deformable_ants(warp_img, outfolder, outdataset):
    #outdir = os.path.join('output', outdataset, outfolder, 'Jacobian_Deformable')
    #outname = outdir + '/JacobianDeformable_' + warp_img.split('/')[-1].split('Warp')[0] + '.nii.gz'
    #print(outname)
    #jacobian_command = "CreateJacobianDeterminantImage 3 " + str(warp_img) + " " + str(outname) + " 0 1"
    #subprocess.call(jacobian_command.split(" "))
    ####################
    outdir = os.path.join('output', outdataset, outfolder, 'Log10Jacobian_Deformable')
    outname = outdir + '/Log10JacobianDeformable_' + warp_img.split('/')[-1].split('Warp')[0] + '.nii.gz'
    #print(outname)
    jacobian_command = "CreateJacobianDeterminantImage 3 " + str(warp_img) + " " + str(outname) + " 1 1"
    subprocess.call(jacobian_command.split(" "))

In [ ]:
outfolders = [
            #"scenario1-0.1",
            #"scenario2-0.1",
            #"scenario3-0.1",
            #"scenario1-8.0",
            #"scenario2-8.0",
            #"scenario3-8.0",
            #"scenario1-8.0-inv",
            #"scenario1-8.0-inv-ddf",
            #"scenario-sym",
            #"scenario-sym-noise",
            #"test-convert",
]
models = [
    #"test-new/scenario1_old_1.0-0.0-0.1.pth",
    #"test-new/scenario2_old_1.0-0.0-0.1.pth",
    #"test-new/scenario3_old_1.0-0.0-0.1.pth",
    #"test-new/scenario1_old_1.0-0.0-8.0.pth",
    #"test-new/scenario2_old_1.0-0.0-8.0.pth",
    #"test-new/scenario3_old_1.0-0.0-8.0.pth",
    #"test-new/scenario1_inv_1.0-0.0-8.0.pth",
    #"test-new/scenario1_inv_ddf_1.0-0.0-0.0.pth",
    #"scenario1_sym_1.0-0.0-8.0.pth",
    #"scenario1_sym_noise_1.0-0.0-8.0.pth",
    #"gin_sym_ft_1.0-0.0-8.0.pth"
]
for i, model in enumerate(models):
    #outdataset = 'Feminad'
    outdataset = 'GIN'
    
    outfolder = outfolders[i]
    mris = sorted(glob(os.path.join('dataset3', outdataset, 'MRI', "*_affine.nii.gz")))
    atlas_name = "dataset3/Atlas/P56_Atlas_128_norm_id.nii.gz"
    ants_warp = sorted(glob(os.path.join("output", "Feminad", "ANTS", "DeformableWarp", "*.nii.gz")))
    affine = nib.load(ants_warp[0]).affine
    header = nib.load(ants_warp[0]).header 
    #affine = nib.load(atlas_name).affine
    #header = nib.load(atlas_name).header 

    for i, mri in enumerate(mris):
        outname = "output/" + outdataset + "/" + outfolder + "/MRI_N4_Registration_Deformable/" + mri.split('/')[-1].split('.')[0]
        if "newmodel" in model:
            newmodel = True
        else:
            newmodel = False
            
        if "sym" in model:
            sym = True
            pred_images, ddfs = reg_mri.main(model, mri, outname, False, True, "local", newmodel=newmodel, sym=sym)
            pred_image = pred_images[0]
            predsym_image = pred_images[1]
        else:
            sym = False
            pred_image, ddfs = reg_mri.main(model, mri, outname, False, True, "local", newmodel=newmodel, sym=sym)

        moving_image = ddfs[0].cpu()

        deformable_ddf = ddfs[1].cpu()
        deformable_ddf = deformable_ddf.cpu().numpy()
        deformable_ddf = np.transpose(deformable_ddf, (2, 3, 4, 0, 1)) 
        deformable_ddf = deformable_ddf * [-1, -1, 1]
        deformable_image = nib.Nifti1Image(deformable_ddf, affine, header)
        outname_deformable = "output/" + outdataset + "/" + outfolder + "/DeformableWarp/" + mri.split('/')[-1].split('.')[0] + "_warp.nii.gz"
        nib.save(deformable_image, outname_deformable)
        
        if sym:
            deformable_sym_ddf = ddfs[2].cpu()
            deformable_sym_ddf = deformable_sym_ddf.cpu().numpy()
            deformable_sym_ddf = np.transpose(deformable_sym_ddf, (2, 3, 4, 0, 1)) 
            deformable_sym_ddf = deformable_sym_ddf * [-1, -1, 1]
            deformable_sym_image = nib.Nifti1Image(deformable_sym_ddf, affine, header)
            outname_sym_deformable = "output/" + outdataset + "/" + outfolder + "/DeformableWarp/" + mri.split('/')[-1].split('.')[0] + "_invwarp.nii.gz"
            nib.save(deformable_sym_image, outname_sym_deformable)
        
    mridir = 'output/' + outdataset + '/' + outfolder + '/'
    #affine_warps = sorted(glob(os.path.join(mridir, 'AffineWarp', "*.nii.gz")))
    deformable_warps = sorted(glob(os.path.join(mridir, 'DeformableWarp', "*_warp.nii.gz")))
    for i in range(len(deformable_warps)):
        #getLog10Jacobian_affine_ants(affine_warps[i], outfolder)    
        #print(deformable_warps[i])
        getLog10Jacobian_deformable_ants(deformable_warps[i], outfolder, outdataset)  

In [ ]:
#def getLog10Jacobian_affine_ants(warp_img, outfolder):
#    outdir = os.path.join('output', 'Feminad', outfolder, 'Log10Jacobian_Affine')
#    outname = outdir + '/Log10JacobianAffine_' + warp_img.split('/')[-1].split('Warp')[0] + '.nii.gz'
#    jacobian_command = "CreateJacobianDeterminantImage 3 " + str(warp_img) + " " + str(outname) + " 1 1"
#    subprocess.call(jacobian_command.split(" "))

In [ ]:
#def getLog10Jacobian_affinedeformable_ants(aff_log10jac, def_log10jac, outfolder):
#    outdir = os.path.join('output', 'Feminad', outfolder, 'Log10Jacobian_AffineDeformable')
#    outname = outdir + '/Log10JacobianAffineDeformable' + aff_log10jac.split('/')[-1].split('Log10JacobianAffine')[-1]
#    combine_command = "ImageMath 3 " + str(outname) + " + " + str(aff_log10jac) + " " + str(def_log10jac)
#    subprocess.call(combine_command.split(" "))

In [ ]:
#aff_log10jacs = sorted(glob(os.path.join(mridir, 'Log10Jacobian_Affine', "*.nii.gz")))
#def_log10jacs = sorted(glob(os.path.join(mridir, 'Log10Jacobian_Deformable', "*.nii.gz")))
#for i in range(len(aff_log10jacs)):
#    getLog10Jacobian_affinedeformable_ants(aff_log10jacs[i], def_log10jacs[i], outfolder)